In [29]:
!pip install efficientnet_pytorch


  Using cached efficientnet_pytorch-0.7.1.tar.gz (21 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16464 sha256=618a8e5f8c60ec18e3c3889c72bdecc2d30edd53668190e816c121aee20ad45e
  Stored in directory: c:\users\sneha\appdata\local\pip\cache\wheels\9c\3f\43\e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet_pytorch


In [ ]:
!pip uninstall -y numpy scipy scikit-learn
!pip install --no-cache-dir numpy scipy scikit-learn


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: scipy 1.15.2
Uninstalling scipy-1.15.2:
  Successfully uninstalled scipy-1.15.2
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1


You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.


In [31]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from tqdm import tqdm
import numpy as np
import os
from PIL import ImageFile

# Skip truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Clear GPU memory
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define dataset paths
data_dir = r"C:\Users\sneha\Downloads\hi\hi"
train_dir = f"{data_dir}/train"
val_dir = f"{data_dir}/val"

# 🔹 Improved Data Augmentation to Avoid Overfitting
transform = transforms.Compose([
    transforms.RandomResizedCrop(380, scale=(0.6, 1.0)),  
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.2, 0.2), shear=10),
    transforms.GaussianBlur(3),  
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

# Create dataloaders
batch_size = 16  # Efficient batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# 🔹 Load Pre-trained EfficientNet-B4
model = EfficientNet.from_pretrained("efficientnet-b4")

# 🔹 Add Dropout to Reduce Overfitting
model._dropout = nn.Dropout(p=0.5)  

# 🔹 Unfreeze More Layers for Fine-Tuning
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers
for param in model._blocks[-6:].parameters():  
    param.requires_grad = True  # Unfreeze last 6 blocks

# 🔹 Modify Classifier Layer for Binary Classification
num_features = model._fc.in_features
model._fc = nn.Linear(num_features, 2)  
model = model.to(device)

# 🔹 Weighted Loss Function for Class Imbalance
class_counts = [500, 200]  
class_weights = torch.tensor([1.0 / c for c in class_counts], device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# 🔹 Optimizer and Scheduler (Lower Learning Rate)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-7)

# Enable mixed precision training
scaler = torch.amp.GradScaler()

# Gradient accumulation steps
accum_steps = 4  

# 🔹 Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    best_acc = 0.0
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        tqdm_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}")

        optimizer.zero_grad()
        for i, (inputs, labels) in tqdm_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(inputs)
                loss = criterion(outputs, labels) / accum_steps

            scaler.scale(loss).backward()
           
            if (i + 1) % accum_steps == 0:  
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            running_loss += loss.item() * accum_steps
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            tqdm_bar.set_postfix(loss=running_loss / (i + 1), acc=100 * correct_train / total_train)

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct_train / total_train

        # 🔹 Validation Phase
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_loss = running_val_loss / len(val_loader)
        val_acc = 100 * correct_val / total_val

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        # 🔹 Save Best Model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_efficientnet_deepfake.pth")

        scheduler.step()  

        print(f"\nEpoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%\n")

    return history

# 🔹 Train the Model
history = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

# Save Final Model
torch.save(model.state_dict(), "efficientnet_deepfake_classifier.pth")

# 🔹 Evaluation Function
def evaluate(model, dataloader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:\n", cm)
    print("Classification Report:\n", classification_report(y_true, y_pred))
    print("F1 Score:", f1_score(y_true, y_pred, average="weighted"))

# 🔹 Evaluate Model
evaluate(model, val_loader)

# 🔹 Test Accuracy Function
def test_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# 🔹 Compute Test Accuracy
test_accuracy(model, val_loader)


ModuleNotFoundError: No module named 'numpy.char'

In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
import os

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the same image transformations used during training
transform = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the trained model
model = EfficientNet.from_name("efficientnet-b4")
num_features = model._fc.in_features
model._fc = nn.Linear(num_features, 2)  # Binary classification

# Load trained weights
model.load_state_dict(torch.load(r"/home/lab-09/Downloads/Deepfake-Detection/model/efficientnet_deepfake_classifier.pth", map_location=device))
model.to(device)
model.eval()  # Set the model to evaluation mode

# Define class labels
class_labels = {0: "Fake", 1: "Real"}

# Function to predict if an image is fake or real
def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: File '{image_path}' not found.")
        return
   
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        confidence = torch.nn.functional.softmax(output, dim=1)[0][predicted].item() * 100

    # Print result
    print(f"Prediction: {class_labels[predicted.item()]} (Confidence: {confidence:.2f}%)")

# Test with an image
image_path = r"/home/lab-09/Downloads/IMG_20250308_133211.jpg" # Change this to the path of your test image
predict_image(image_path)



Using device: cuda
Prediction: Real (Confidence: 68.52%)


In [11]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from tqdm import tqdm
import numpy as np
import os
from PIL import ImageFile
import cv2
import shap
import matplotlib.pyplot as plt
from tf_keras_vis.saliency import Gradcam
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore

# Skip truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Clear GPU memory
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define dataset paths
data_dir = r"C:\Users\sneha\Downloads\hi\hi"
train_dir = f"{data_dir}/train"
val_dir = f"{data_dir}/val"

# Data Augmentation
transform = transforms.Compose([
    transforms.RandomResizedCrop(380, scale=(0.6, 1.0)),  
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.2, 0.2), shear=10),
    transforms.GaussianBlur(3),  
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

# Create dataloaders
batch_size = 16  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# Load Pre-trained EfficientNet-B4
model = EfficientNet.from_pretrained("efficientnet-b4")
model._dropout = nn.Dropout(p=0.5)  

# Unfreeze last layers
for param in model.parameters():
    param.requires_grad = False  
for param in model._blocks[-6:].parameters():  
    param.requires_grad = True  

# Modify Classifier Layer for Binary Classification
num_features = model._fc.in_features
model._fc = nn.Linear(num_features, 2)  
model = model.to(device)

# Weighted Loss Function for Class Imbalance
class_counts = [500, 200]  
class_weights = torch.tensor([1.0 / c for c in class_counts], device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Optimizer and Scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-7)

scaler = torch.amp.GradScaler()
accum_steps = 4  

# Train Model
history = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)
torch.save(model.state_dict(), "efficientnet_deepfake_classifier.pth")

def visualize_gradcam(model, image):
    gradcam = Gradcam(model, model_modifier=ReplaceToLinear(), clone=True)
    score = CategoricalScore([1])  
    heatmap = gradcam(score, image)
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap[0]), cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

def test_on_new_dataset(model, dataset_path):
    test_transform = transforms.Compose([
        transforms.Resize((380, 380)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    test_dataset = datasets.ImageFolder(root=dataset_path, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    evaluate(model, test_loader)

print("Testing on new dataset...")
test_on_new_dataset(model, "path/to/new/dataset")


ModuleNotFoundError: No module named 'torchvision'

In [3]:
import os
import random

# Set the path to your dataset folder
dataset_path = r"C:\Users\sneha\Downloads\hi\hi\Fake"

# Get a list of all image files in the dataset folder
image_files = [f for f in os.listdir(dataset_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

# Ensure there are at least 20,000 images before attempting to delete
if len(image_files) < 20000:
    print("Not enough images in the dataset to delete 20,000.")
else:
    # Randomly select 20,000 images to delete
    images_to_delete = random.sample(image_files, 11193)

    # Delete the selected images
    for image in images_to_delete:
        image_path = os.path.join(dataset_path, image)
        os.remove(image_path)
    
    print(f"Deleted {len(images_to_delete)} images successfully.")


Deleted 11193 images successfully.


In [9]:
pip uninstall torch torchvision torchaudio -y
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


SyntaxError: invalid syntax (3084758649.py, line 1)